In [ ]:
from pathlib import Path
from typing import Union, Sequence

import nibabel as nib
import numpy as np
import pandas as pd
from brainstat.datasets import fetch_template_surface
from brainstat.stats.SLM import SLM
from brainstat.stats.terms import FixedEffect, MixedEffect

In [ ]:
def load_thickness(
    mics_directory: Path, subjects: Union[str, Sequence[str]], space: str = "fsaverage5"
) -> np.ndarray:
    """"""

    if isinstance(subjects, str):
        subjects = [subjects]

    for i in range(len(subjects)):
        thickness_dir = (
            mics_directory
            / "derivatives"
            / "micapipe"
            / subjects[i]
            / "ses-01"
            / "anat"
            / "surfaces"
            / "morphology"
        )
        thickness_files = [
            thickness_dir / f"{s}_ses-01_space-{space}_desc-{hemi}_thickness_10mm.mgh"
            for hemi in ["lh", "rh"]
        ]
        thickness_data = [nib.load(x).get_data() for x in thickness_files]
        if i == 0:
            thickness = np.zeros(thickness_data[0].shape, len(subjects))
        thickness[:, i] = np.concatenate(thickness_data, axis=0)
    return thickness    

In [ ]:
mics_directory = Path("/data/mica3/BIDS_MICs_release_v1")
micapipe = mics_directory / "derivatives" / "micapipe"
subjects = [x.name for x in micapipe.iterdir() if x.is_dir()]

subject_encoding_file = "/data_/mica3/BIDS_MICs/MICs_rename-equivalence_latest.csv"
subject_encoding = pd.read_csv(subject_encoding_file, sep=',')
subject_encoding["Original"] = "sub-" + subject_encoding["Original"]

cortical_thickness = load_thickness(mics_directory, subjects, space="fsaverage5")
indices = [np.argwhere(subject_encoding["Original"] == x) for x in subjects]
cortical_thickness = cortical_thickness[:, indices]

fsaverage5 = fetch_template_surface("fsaverage5")

behavioral_file = "/data/mica3/BIDS_MICs/MICs_release/rawdata/participants.tsv"
behavioral = pd.read_csv(behavioral_file, sep='\t')

age = FixedEffect(behavioral["age"])
sex = FixedEffect(behavioral["sex"])

# Simple model
model = age + sex
slm = SLM(model, -age, surf=fsaverage5, correction="rft")
slm.fit(cortical_thickness)

# Interaction model
model_interaction = age + sex + age * sex
slm_interaction = SLM(model_interaction, -age, surf=fsaverage5, correction="rft")
slm_interaction.fit(cortical_thickness)

# Repeated Measures
subject = MixedEffect(SUBJECT_DATA)
model_random = age + sex + subject
slm_random = SLM(model_random, -age, surf=fsaverage5, correction="rft")
slm_random.fit(cortical_thickness)
